In [16]:
import requests
import pandas as pd 
from datetime import datetime

In [17]:
# Conexão com API

NOTION_API_KEY = 'secret_GHlV2yGtPWvW6TYpDCHAebaP0QmUbLxc7kEN9e6iVwj'
headers = {
    'Authorization': f'Bearer {NOTION_API_KEY}',
    'Content-Type': 'application/json',
    'Notion-Version': '2022-06-28',
}

In [18]:
# Substitua 'ID_DO_BANCO_DE_DADOS' pelo ID do banco de dados que você deseja consultar
database_id = 'e51a95d56d594c16ad7370bb669f21e5'

# Faça a solicitação GET para consultar o banco de dados
response = requests.post(f'https://api.notion.com/v1/databases/{database_id}/query', headers=headers)

In [19]:
# Analise a resposta
if response.status_code == 200:
    response_data = response.json()
    
    if 'results' in response_data:
        # Aplicar a normalização aos dados
        df = pd.json_normalize(response_data, 'results')
        
    else:
        print("A resposta não contém dados de resultado.")
else:
    print(f'Erro ao consultar o banco de dados. Código de status: {response.status_code}')

In [20]:

# Função para tratar campos específicos
def extrair_valores(valores):
    if isinstance(valores, list):
        # Verifique se a lista não está vazia
        if valores:
            return valores[0].get('text', {}).get('content')
    return None

In [21]:
# Aplique a função à coluna 'properties.Nome.title'
coluna_alvo = 'properties.Name.title'

if coluna_alvo in df.columns:
    df[coluna_alvo] = df[coluna_alvo].apply(extrair_valores)
else:
    print(f"A coluna '{coluna_alvo}' não existe no DataFrame.")

In [22]:
coluna_alvo = 'properties.Texto.rich_text'

if coluna_alvo in df.columns:
    df[coluna_alvo] = df[coluna_alvo].apply(extrair_valores)
else:
    print(f"A coluna '{coluna_alvo}' não existe no DataFrame.")

In [23]:
# Função para extrair o 'id' e 'name' do dicionário
def extrair_id_name(valores):
    if isinstance(valores, list):
        # Verifique se a lista não está vazia
        if valores:
            id_value = valores[0].get('id')
            name_value = valores[0].get('name')
            return id_value, name_value
    return None, None

# Coluna alvo
coluna_alvo = 'properties.Tags.multi_select'

# Verifique se a coluna existe
if coluna_alvo in df.columns:
    # Aplique a função à coluna
    df[['Tag Id', 'Tag Name']] = df[coluna_alvo].apply(extrair_id_name).apply(pd.Series)
else:
    print(f"A coluna '{coluna_alvo}' não existe no DataFrame.")

A coluna 'properties.Tags.multi_select' não existe no DataFrame.


In [24]:
# Função para formatar uma única data
def formatar_data(df):
    formato_com_horario = "%Y-%m-%dT%H:%M:%S.%f%z"
    formato_somente_data = "%Y-%m-%d"

    if isinstance(df, str):
        try:
            # Tenta converter com horário
            data_formatada = datetime.strptime(df, formato_com_horario)
            return data_formatada.strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            try:
                # Tenta converter somente a data
                data_formatada = datetime.strptime(df, formato_somente_data)
                return data_formatada.strftime("%Y-%m-%d")
            except ValueError:
                return df  # Se não for possível formatar, retorne o valor original
    elif pd.isna(df):
        return None  # Se for NaN, retorne None
    else:
        return df  # Se não for uma string, retorne o valor original

# Aplicar a função a cada coluna de data
colunas_de_data = ['properties.Data Criação.date.start', 'properties.Data Criação.date.end']

for coluna in colunas_de_data:
    if coluna in df.columns:
        df[coluna] = df[coluna].apply(formatar_data)
    else:
        print(f"A coluna '{coluna}' não existe no DataFrame.")


In [25]:
columns_of_interest = [
    'url',
    'created_time',
    'last_edited_time',
    'created_by.id',
    'last_edited_by.id',
    'parent.database_id',
    'Tag Id',
    'Tag Name',
    'properties.Nome.title',
    'properties.Status.status.name',
    'properties.Texto.rich_text',
    'properties.Data Criação.date.start',
    'properties.Data Criação.date.end',
    'icon.emoji'
]

# Verifique e crie as colunas ausentes
for coluna in columns_of_interest:
    if coluna not in df.columns:
        df[coluna] = None

# Crie um novo DataFrame com as colunas de interesse
df_filtered = df[columns_of_interest]

In [26]:
df_filtered.to_excel('C:/Users/Lucas Gomes/Downloads/api_notion.xlsx',index=False)